In [2]:
# Cell 2: imports & load dataset
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

CSV_PATH = r"C:\Users\vishn\Documents\HEBA\data\fer2013.csv"  # your path

df = pd.read_csv(CSV_PATH)
print(df.head())
print("Total samples:", len(df))
print("Emotion distribution:\n", df['emotion'].value_counts())


   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training
Total samples: 35887
Emotion distribution:
 emotion
3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: count, dtype: int64


In [3]:
# Cell 3: Create PyTorch Dataset loader
from torchvision import transforms

class FERDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        pixels = np.array(self.df.iloc[idx]['pixels'].split(), dtype=np.uint8)
        img = pixels.reshape(48,48)
        
        if self.transform:
            img = self.transform(img)
        
        label = int(self.df.iloc[idx]['emotion'])
        return img, label

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [4]:
# Cell 4: split dataset
train_df, test_df = train_test_split(df, test_size=0.15, stratify=df['emotion'], random_state=42)

train_data = FERDataset(train_df, transform)
test_data  = FERDataset(test_df, transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
]))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=64, shuffle=False)

print("Training Samples:", len(train_data))
print("Testing Samples:", len(test_data))


Training Samples: 30503
Testing Samples: 5384


In [5]:
# Cell 5: Basic CNN suitable for 48x48 images
import torch.nn as nn
import torch.nn.functional as F

class EmotionCNN(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(64*12*12, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        return self.fc2(x)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = EmotionCNN().to(device)
print(model)


EmotionCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [7]:
# Cell 6: Training with Loss + Accuracy + Progress Bar
from tqdm import tqdm   # ⭐ Progress bar (pip install tqdm if not installed)

epochs = 25
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", colour="green")

    for imgs, labels in progress:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # ---------- Accuracy Calculation ----------
        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        # Update progress bar live
        progress.set_postfix({
            "Loss": f"{total_loss/len(train_loader):.4f}",
            "Accuracy": f"{(total_correct/total_samples)*100:.2f}%"
        })

    print(f"\n✔ Epoch {epoch+1}/{epochs} Completed | "
          f"Loss: {total_loss/len(train_loader):.4f} | "
          f"Accuracy: {(total_correct/total_samples)*100:.2f}%\n")


Epoch 1/25: 100%|██████████| 477/477 [00:26<00:00, 17.95it/s, Loss=1.4045, Accuracy=46.46%]



✔ Epoch 1/25 Completed | Loss: 1.4045 | Accuracy: 46.46%



Epoch 2/25: 100%|██████████| 477/477 [00:28<00:00, 16.72it/s, Loss=1.3287, Accuracy=48.98%]



✔ Epoch 2/25 Completed | Loss: 1.3287 | Accuracy: 48.98%



Epoch 3/25: 100%|██████████| 477/477 [00:28<00:00, 16.93it/s, Loss=1.2728, Accuracy=51.26%]


...


✔ Epoch 23/25 Completed | Loss: 0.7824 | Accuracy: 70.29%



Epoch 24/25: 100%|██████████| 477/477 [00:24<00:00, 19.31it/s, Loss=0.7657, Accuracy=70.62%]



✔ Epoch 24/25 Completed | Loss: 0.7657 | Accuracy: 70.62%



Epoch 25/25: 100%|██████████| 477/477 [00:25<00:00, 18.87it/s, Loss=0.7485, Accuracy=70.86%]


✔ Epoch 25/25 Completed | Loss: 0.7485 | Accuracy: 70.86%



In [ ]:
# Cell 7: testing accuracy
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"\nTest Accuracy: {correct/total*100:.2f}%")
